# Imports

In [42]:
import os
import pandas as pd
import numpy as np
import geopandas as gpd
import math

from tensorflow import keras
import tensorflow as tf
from sklearn.linear_model import LinearRegression

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.ticker import MaxNLocator
from matplotlib.patches import Wedge
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import matplotlib.patches as mpatches

from shapely.geometry import MultiPolygon, box
from shapely import affinity
from shapely.ops import transform , unary_union, cascaded_union, linemerge
from shapely.wkt import loads

import random
from PIL import Image

# Definitions

In [43]:
# paths
path_CS_png = f'C:/Users/Name/OneDrive - Delft University of Technology/Building Technology-Thesis/CaseStudy/CaseStudy2/RoomFeature_png'
path_CS_npy = f'C:/Users/Name/OneDrive - Delft University of Technology/Building Technology-Thesis/CaseStudy/CaseStudy2/RoomFeature_npy'

In [44]:
# Define a custom function to create the desired string
def create_filename(row):
    return f"{row['apartment_id']}_{row['area_id']}.npy"

## Def room image feature

In [45]:
# Define the desired sort order as a list with corresponding colours
# Create a dictionary to map categories to their corresponding colors
category_to_color = {'ROOM':            '#F7D08A',  # sunset / light yellow
                     'WALL':            '#F6F5F4',  # smoke white
                     'DOOR':            '#E5DEDC',  #timerwolf
                     'OUTSIDE_DOOR':    '#94BDAA',  #cambridge blue
                     'WINDOW':          '#B5E3F1',  #light blue
                    }

# Extract the categories from the data and map them to their corresponding colors
categories = list(category_to_color.keys())
colors = [category_to_color[category] for category in categories]

# Create a color map from the used colors
color_map = ListedColormap(colors)

In [46]:
def bbox_rect(gdf_room):
  #Create a dataframe with only rooms and create a small buffer for geometry
  gdf_room_area = gdf_room[gdf_room['entity_subtype'] == 'ROOM']

  #Take the unary union of the buffered polygons and create the minimum rotated rectangle
  poly_rooms = gdf_room_area["geometry"].unary_union
  rect_rooms = poly_rooms.minimum_rotated_rectangle

  #Calculate the current dimensions of the rectangle in meters
  x_info, y_info = rect_rooms.exterior.xy
  x_distance = max(x_info) - min(x_info)
  y_distance = max(y_info) - min(y_info)

  #Scale the rectangle tothe specified unit size = 15x15
  xfact = 15 / x_distance
  yfact = 15 / y_distance
  rect_rooms_buffered = affinity.scale(rect_rooms, xfact=xfact, yfact=yfact)

  #Create a bounding box around the rect_rooms_buffered with size 15x15
  rect = box(*rect_rooms_buffered.bounds)
  gdf_rect = gpd.GeoDataFrame(geometry=[rect])

  # Calculate the environmental circles
  rect_centroid = rect.centroid
  img_centre = (rect_centroid.x, rect_centroid.y)

  return gdf_rect, img_centre

In [47]:
def create_image_feature(df_casestudy, path_CS_png, path_CS_npy):
  #Create geoseries with polygons -> change wkt to polygon
  df_casestudy['geometry'] = df_casestudy['geometry'].apply(loads)
  gdf_casestudy = gpd.GeoDataFrame(df_casestudy, geometry='geometry', crs=None)

  # Exclude values in unique_plotted_apartment_ids from unique_app_ids
  app_ids_casestudy = df_casestudy['apartment_id'].unique()
  app_ids_casestudy = list(app_ids_casestudy)
  
  # run loop to generate room images
  for app_id in app_ids_casestudy:
  #for app_id in app_ids:
    #Get dataframe of apartment in which room lies
    gdf_apartment = gdf_casestudy.loc[(gdf_casestudy["apartment_id"]== app_id)]
    gdf_apartment_doors = gdf_casestudy.loc[(gdf_casestudy["apartment_id"] == app_id) & (gdf_casestudy["entity_subtype"] == 'DOOR')]
    
    gdf_outdoor_space = gdf_casestudy.loc[(gdf_casestudy["apartment_id"] == app_id) & (gdf_casestudy["entity_subtype"] == 'FLOOR_OUTDOOR_SPACE')]
    gdf_floor = gdf_casestudy.loc[(gdf_casestudy["apartment_id"] == app_id) & (gdf_casestudy["entity_subtype"] == 'FLOOR')]
    #print(app_id)

  #for area_id in apartment:
    for area_id in gdf_apartment['area_id'].unique():
      if area_id != 0:
        #locate the specific apartement in the dataframe
        gdf_room = gdf_apartment.loc[gdf_apartment["area_id"]== area_id]
        room_type = gdf_room['entity_subtype'].iloc[0]

        if room_type in ['BEDROOM', 'ROOM', 'KITCHEN', 'DINING', 'LIVING_ROOM']:
          # check if the dataframe is not empty
          if not gdf_room.empty:
            #Generate the full room geometry
            # Check if 'door_connection1' or 'door_connection2' is equal to area_id
            mask = (gdf_apartment_doors['door_connection1'] == area_id) | (gdf_apartment_doors['door_connection2'] == area_id)

            # Add the matching rows to gdf_room_doors
            gdf_room_doors = gdf_apartment_doors[mask]
            gdf_room = pd.concat([gdf_room, gdf_room_doors], ignore_index=False)
            
            # Update 'entity_subtype' to 'ROOM' when it's not equal to 'window', 'door', or 'outside_door'
            gdf_room.loc[~gdf_room['entity_subtype'].isin(['WINDOW', 'DOOR', 'OUTSIDE_DOOR']), 'entity_subtype'] = 'ROOM'

            # Set the figure size in inches to match the desired pixel size (224x224)
            fig, ax = plt.subplots(figsize=(10, 10), dpi=100)

            #plot the floor of the apartment
            gdf_floor.plot(ax=ax, color='#F6F5F4')
            
            #plot the outdoor spaces
            if not gdf_outdoor_space.empty:
              gdf_outdoor_space.plot(ax=ax, color='#ECEAE9')

                  #plot the room geomerty 
            gdf_room.plot(column="entity_subtype", cmap=color_map, legend=False, ax=ax, categories=categories)
            
            #Find boundary rectangle of 15x15 meters
            gdf_rect, img_centre = bbox_rect(gdf_room)
            #gdf_rect.boundary.plot(ax=ax, color='grey', linestyle=(5, (10, 3)), linewidth=1)

            #Get information about view landscape layer
            view_nature = gdf_apartment['view_landscape_nature_mean'].iloc[0]    
            view_urban = gdf_apartment['view_landscape_urban_mean'].iloc[0]
            view_landscape = view_nature + view_urban
            
            # Only plot the diagram if the landscape layer is visible according to research 
            if view_landscape > 0.477: 
              #print(view_landscape)
              #Get data for circle
              radius = view_landscape/2.5
              split_angle = ((view_nature/view_landscape) * 360) + 90

              #print(f'total: {view_landscape}, nature: {view_nature} - {view_nature/view_landscape}%, urban: {view_urban}')

              #Plot the view_landscape circle
              circle_white = Wedge(center=img_centre, r=radius, theta1=0, theta2=360, facecolor='white', alpha=0.3)
              ax.add_patch(circle_white)
              circle_nature = Wedge(center=img_centre, r=radius, theta1=90, theta2=split_angle, facecolor='#AACFB5', alpha=0.4)
              ax.add_patch(circle_nature)
              circle_urban = Wedge(center=img_centre, r=radius, theta1=split_angle, theta2=450, facecolor='#CF8BA3', alpha=0.4)
              ax.add_patch(circle_urban)

              #set image settings
              bounds = gdf_rect.total_bounds
              ax.set_xlim(bounds[0], bounds[2])
              ax.set_ylim(bounds[1], bounds[3])
              ax.axis('off')

              #Remove the borders of the plot
              ax.spines['top'].set_visible(False)
              ax.spines['right'].set_visible(False)
              ax.spines['bottom'].set_visible(False)
              ax.spines['left'].set_visible(False)

              # Set the filename based on the site_id, app_id, and area_id
              area_id1 = area_id.astype(int)
              filename_png = f"{app_id}_{area_id1}.png"
              filename_npy = f"{app_id}_{area_id1}.npy"

              plt.savefig(os.path.join(path_CS_png, filename_png), bbox_inches='tight', pad_inches=0, dpi=200)
              plt.close(fig)

              # Load the PNG image and save it as a NumPy array in the same folder
              image_path = os.path.join(path_CS_png, filename_png)
              
              # Open the image file using a with statement to ensure it's closed properly
              with Image.open(image_path) as image:
                  # Resize the image to the needed size
                  new_size = (224, 224)
                  image = image.convert('RGB')
                  resized_image = image.resize(new_size)

              #create and save the numpy array
              image_array = np.array(resized_image)
              numpy_save_path = os.path.join(path_CS_npy, filename_npy)
              np.save(numpy_save_path, image_array)

## Create features list

In [48]:
def process_images(file_names, image_folder_path, image_feature, df_image_feature):
    # Iterate over all images in the folder
    for filename in file_names:
        # Construct the full file path
        file_path = os.path.join(image_folder_path, filename)

        # Load the image data as a NumPy array and append it to the list
        loaded_array = np.load(file_path)
        image_feature.append(loaded_array)

        # Extract app_id and area_id from the filename using string splitting
        app_id, area_id = filename.split('_')[0], filename.split('_')[1].split('.')[0]

        # Add data to the DataFrame for each iteration
        df_image_feature = df_image_feature.append({
            'apartment_id': app_id,
            'area_id': int(area_id),
            'filename': filename
        }, ignore_index=True)

    return image_feature, df_image_feature

In [49]:
def make_features_labels(df_casestudy, path_CS_npy):
    # Apply the function to create the new column
    df_casestudy_rooms = df_casestudy.loc[~df_casestudy['entity_subtype'].isin(['WINDOW', 'OUTSIDE_DOOR', 'DOOR', 'OUTDOOR_SPACE', 'CORRIDOR', 'BATHROOM','FLOOR', 'FLOOR_OUTDOOR_SPACE'])]

    df_casestudy_rooms['filename'] = df_casestudy_rooms.apply(create_filename, axis=1)
    filename_list = df_casestudy_rooms['filename'].tolist()
    
    # Create an empty DataFrame with columns
    columns = ['apartment_id', 'area_id', 'filename']
    df_image_feature = pd.DataFrame(columns=columns)
    image_feature = []

    #load the npy file and append to df
    image_feature, df_image_feature = process_images(filename_list, path_CS_npy, image_feature, df_image_feature)

    # Add info to list
    app_ids = df_image_feature['apartment_id'].tolist()
    unique_app = np.unique(app_ids)
    area_ids = df_image_feature['area_id'].tolist()
    used_files_id = df_image_feature['filename'].tolist()

    # create a dataframe with only the features and labels out of the rooms with images in folder
    df_info_used = df_casestudy_rooms.loc[(df_casestudy_rooms['apartment_id'].isin(unique_app))]

    # Find the landscape values
    df_info_used['view_landscape_values'] = df_info_used['view_landscape_nature_mean'] + df_info_used['view_landscape_urban_mean']

    # min and max values from dataset during training
    min_elevation = -8.7
    max_elevation = 56.97999999999999
    max_WFR = 1.071

    # Normalise the features
    df_info_used['elevation_normalized'] = (df_info_used['elevation'] - min_elevation) / (max_elevation - min_elevation)
    df_info_used['window_floor_ratio_normalized'] = df_info_used['window_floor_ratio'] / max_WFR
    df_info_used['window_floor_ratio_normalized'].fillna(0, inplace=True)

    #Make list with the features
    combined_features = []
    entity_subtypes = []

    for i in range(len(used_files_id)):
        app_id = app_ids[i]
        area_id = int(area_ids[i])

        feature_elevation = df_info_used.loc[(df_info_used['apartment_id'] == app_id) & (df_info_used['area_id'] == area_id), 'elevation_normalized'].values[0]
        feature_windowfloor = df_info_used.loc[(df_info_used['apartment_id'] == app_id) & (df_info_used['area_id'] == area_id), 'window_floor_ratio_normalized'].values[0]

        entity_subtype = df_info_used.loc[(df_info_used['apartment_id'] == app_id) & (df_info_used['area_id'] == area_id), 'entity_subtype'].values[0]

        # Create a list with the two labels for each data point and append it to combined_labels
        combined_features.append([feature_elevation, feature_windowfloor])
        entity_subtypes.append(entity_subtype)

    images = np.array(image_feature)
    num_features = np.array(combined_features)

    return app_ids, area_ids, used_files_id, entity_subtypes, images, num_features

## post-process predictions

In [50]:
def unnormalise_predictions(df_casestudy, pred_daylight, pred_view, app_id, area_id): 
    max_daylight_values = [1688, 1472, 1441]
    daylight_values = pred_daylight * max_daylight_values
    daylight_values = np.round(daylight_values, 0)

    max_view_values = [6.87581326693448, 5.42916079787956]
    view_values = pred_view * max_view_values
    view_values = np.round(view_values, 2)

    # Select only living spaces and outdoor spaces and select specific columns
    df_casestudy_rooms = df_casestudy.loc[~df_casestudy['entity_subtype'].isin(['WINDOW', 'OUTSIDE_DOOR', 'DOOR', 'CORRIDOR', 'BATHROOM','FLOOR', 'FLOOR_OUTDOOR_SPACE'])]
    df_casestudy_rooms.loc[:, 'view_landscape'] = df_casestudy_rooms['view_landscape_nature_mean'] + df_casestudy_rooms['view_landscape_urban_mean']
    selected_columns = ['apartment_id', 'area_id', 'entity_subtype', 'elevation', 'height', 'orientation', 'window_area', 'window_floor_ratio', 'view_landscape']
    df_casestudy_rooms = df_casestudy_rooms[selected_columns]

    # Create a DataFrame for the daylight predictions with 'pred_app_id' and 'pred_area_id'
    df_predictions = pd.DataFrame({
        'apartment_id': app_id,
        'area_id': area_id,
        'daylight_Mar_norm': pred_daylight[:, 0],
        'daylight_Jun_norm': pred_daylight[:, 1],
        'daylight_Dec_norm': pred_daylight[:, 2],
        'view_ground_norm': pred_view[:, 0],
        'view_sky_norm': pred_view[:, 1],
        'daylight_Mar': daylight_values[:, 0],
        'daylight_Jun': daylight_values[:, 1],
        'daylight_Dec': daylight_values[:, 2],
        'view_ground': view_values[:, 0],
        'view_sky': view_values[:, 1]
    })

    # Merge the existing df_casestudy with the df_predictions using 'app_id' and 'area_id'
    df_pred_design = df_casestudy_rooms.merge(df_predictions, on=['apartment_id', 'area_id'], how='left')

    return daylight_values, view_values, df_pred_design

## Performance levels

### Daylight

In [51]:
def find_daylight_level(daylight_median, room_type):
    if daylight_median >= 750:
        daylight_level = 'high'
    
    elif daylight_median >= 500:
        daylight_level = 'med'
    
    elif daylight_median >= 300:
        daylight_level = 'low'
    
    elif daylight_median < 300:
        if room_type in ('ROOM', 'BEDROOM'):
            if daylight_median >= 100:
                daylight_level = 'min'
            else:
                daylight_level = 'insufficient'
        elif room_type in ('LIVING_ROOM', 'DINING'):
            if daylight_median >= 150:
                daylight_level = 'min'
            else:
                daylight_level = 'insufficient'
        elif room_type == 'KITCHEN':
            if daylight_median >= 100:
                daylight_level = 'min'
            else:
                daylight_level = 'insufficient'
    
    else:
        daylight_level = None

    return daylight_level

In [52]:
def find_daylight_label(group):
    daylight_level_median = group['daylight_level_num'].median()
    daylight_level_min = group['daylight_level_num'].min()

    if daylight_level_min == 0:
        daylight_label = 'F'
    elif daylight_level_min == 4:
        daylight_label = 'A'  
    elif daylight_level_min == 1:
        if daylight_level_median >= 2:
            daylight_label = 'D'
        else: 
            daylight_label = 'E'
    elif daylight_level_min == 2:
        if daylight_level_median >= 3:
            daylight_label = 'C'
        elif daylight_level_median >= 2:
            daylight_label = 'D'
        else: 
            daylight_label = 'E'
    elif daylight_level_min == 3:
        if daylight_level_median >= 4:
            daylight_label = 'B'
        elif daylight_level_median >= 3:
            daylight_label = 'C'
        elif daylight_level_median >= 2:
            daylight_label = 'D'
        else: 
            daylight_label = 'E'
    else:
        daylight_label = None
    #print(f'min={daylight_level_min} median={daylight_level_median} label={daylight_label}')
    return daylight_label

In [53]:
# Define the mapping dictionary
mapping_daylight_level = {
    'insufficient': 0,
    'min': 1,
    'low': 2,
    'med': 3,
    'high': 4
}

# Define the mapping dictionary
mapping_daylight_label = {
    'A': 1,
    'B': 0.8,
    'C': 0.6,
    'D': 0.4,
    'E': 0.2,
    'F': 0
}

### View

In [54]:
def count_visible_views(row):
    columns_to_check = ['view_ground', 'view_landscape', 'view_sky']
    count = sum(1 for col in columns_to_check if row[col] >= 0.477)
    return count

def find_view_level(landscape_visible, nr_layers):
    if landscape_visible == 'no':
        view_level = 'insufficient'
    elif nr_layers == 1:
        view_level = 'min'
    elif nr_layers == 2:
        view_level = 'med'
    elif nr_layers == 3:
        view_level = 'high'

    else:
        view_level = None

    return view_level

In [55]:
def find_view_label(group):
    view_level_median = group['view_level_num'].median()
    view_level_min = group['view_level_num'].min()

    if view_level_min == 0:
        view_label = 'E'
    elif view_level_min == 3:
        view_label = 'A'  
    elif view_level_min == 1:
        if view_level_median >= 2:
            view_label = 'C'
        else: 
            view_label = 'D'
    elif view_level_min == 2:
        if view_level_median >= 3:
            view_label = 'B'
        elif view_level_median >= 2:
            view_label = 'C'
        else: 
            view_label = 'D'
    #print(f'min={view_level_min} median={view_level_median} label={view_label}')
    return view_label

In [56]:
# Define the mapping dictionary
mapping_view_level = {
    'insufficient': 0,
    'min': 1,
    'med': 2,
    'high': 3
}

mapping_view_label = {
    'A': 1,
    'B': 0.75,
    'C': 0.5,
    'D': 0.25,
    'E': 0
}

### Orientation

In [57]:
def find_orientation_level(room_type, main_orientation):
    if room_type == 'LIVING_ROOM':
        if main_orientation == 'South-West':
            orientation_level = 'high'
        elif main_orientation in ('West', 'South'):
            orientation_level = 'med'
        elif main_orientation in ('North-West', 'South-East'):
            orientation_level = 'low'
        else: 
            orientation_level = 'min'

    elif room_type == 'DINING':
        if main_orientation in ('South-East', 'South'):
            orientation_level = 'high'
        elif main_orientation in ('East', 'South-West'):
            orientation_level = 'med'
        elif main_orientation in ('North-East', 'West'):
            orientation_level = 'low'
        else: 
            orientation_level = 'min'

    elif room_type == 'KITCHEN':
        if main_orientation == 'East':
            orientation_level = 'high'
        elif main_orientation in ('North-East', 'South-East'):
            orientation_level = 'med'
        elif main_orientation in ('North', 'South'):
            orientation_level = 'low'
        else: 
            orientation_level = 'min'

    elif room_type in ('BEDROOM', 'ROOM'):
        if main_orientation == 'South-East':
            orientation_level = 'high'
        elif main_orientation in ('East', 'South'):
            orientation_level = 'med'
        elif main_orientation in ('North-East', 'South-West'):
            orientation_level = 'low'
        else: 
            orientation_level = 'min'

    elif room_type == 'OUTDOOR_SPACE':
        if main_orientation == 'South':
            orientation_level = 'high'
        elif main_orientation in ('South-East','South-West'):
            orientation_level = 'med'
        elif main_orientation in ('North-East', 'East', 'West', 'North-West'):
            orientation_level = 'low'
        else: 
            orientation_level = 'min'

    else: 
        orientation_level = 'nan'

    return orientation_level

In [58]:
def find_orientation_label(group):
    orientation_level_median = group['orientation_level_num'].median()
    orientation_level_min = group['orientation_level_num'].min()

    if orientation_level_min == 3:
        orientation_label = 'A'  
    elif orientation_level_median >= 3:
        orientation_label = 'B'
    elif orientation_level_median >= 2:
        orientation_label = 'C'
    elif orientation_level_median >= 1:
        orientation_label = 'D'
    else:
        orientation_label = 'E'
    #print(f'min={orientation_level_min} median={orientation_level_median} label={orientation_label}')
    return orientation_label

In [59]:
# Define the mapping dictionary
mapping_orientation_level = {
    'min': 0,
    'low': 1,
    'med': 2,
    'high': 3
}

mapping_orientation_label = mapping_daylight_label

### Overall performance

In [60]:
def find_overall_label_num(day_points, view_points, orientation_points):
    penalty = 1

    if day_points == 0 or view_points == 0 :
        penalty = 0
    
    average = (day_points + view_points + orientation_points) / 3

    overall_label = (average * penalty)

    return overall_label

In [61]:
mapping_overall_label = {
    'A': (1.0, 0.86),   #10
    'B': (0.85, 0.71),  #15
    'C': (0.70, 0.56),  #15
    'D': (0.55, 0.36),  #20
    'E': (0.35, 0.01),  #35
    'F': (0, 0)
}

def find_overall_label(value):
    for letter, (upper, lower) in mapping_overall_label.items():
        if lower <= value <= upper:
            return letter
    return None  # Return None or another value to handle cases outside the defined ranges

### Find performance

In [62]:
def find_performance_levels(df_pred_design):
    # Calculate the median along the specified columns and store it in a new column
    df_pred_design['daylight_median'] = df_pred_design[['daylight_Mar', 'daylight_Jun', 'daylight_Dec']].median(axis=1)

    # Apply the function to the DataFrame and create a new column 'daylight_level'
    df_pred_design['daylight_level'] = df_pred_design.apply(lambda row: find_daylight_level(row['daylight_median'], row['entity_subtype']),axis=1)
    
    df_pred_design['landscape_visible'] = df_pred_design['view_landscape'].apply(lambda x: 'yes' if x >= 0.477 else 'no')
    df_pred_design['visible_view_layers'] = df_pred_design.apply(count_visible_views, axis=1)

    # Apply the function to the DataFrame and create a new column 'view_level'
    df_pred_design['view_level'] = df_pred_design.apply(
        lambda row: find_view_level(row['landscape_visible'], row['visible_view_layers']) 
                    if row['entity_subtype'] != 'OUTDOOR_SPACE' 
                    else None, 
        axis=1)

    # Apply the function to the DataFrame and create a new column 'orientation_level'
    df_pred_design['orientation_level'] = df_pred_design.apply(lambda row: find_orientation_level(row['entity_subtype'], row['orientation']), axis=1)

    # Drop the specified columns
    columns_to_drop = ['daylight_median', 'landscape_visible', 'visible_view_layers']
    df_pred_design = df_pred_design.drop(columns=columns_to_drop, axis=1)

    return df_pred_design

In [63]:
def find_performance_labels(df_pred_design):
    # find the daylight labels for each apartment
    # remap performance levels to numerical values
    df_pred_design['daylight_level_num'] = df_pred_design['daylight_level'].map(mapping_daylight_level)
    df_pred_design['view_level_num'] = df_pred_design['view_level'].map(mapping_view_level)
    df_pred_design['orientation_level_num'] = df_pred_design['orientation_level'].map(mapping_orientation_level)

    # Step 1: Group by 'apartment_id'
    # Apply the function to the DataFrame and create a new column 'daylight_level'
    grouped = df_pred_design.groupby('apartment_id')

    # Step 2: Calculate the median and min values for each apartment group
    daylight_level_median = grouped['daylight_level_num'].median()
    daylight_level_min = grouped['daylight_level_num'].min()
    view_level_median = grouped['view_level_num'].median()
    view_level_min = grouped['view_level_num'].min()
    orientation_level_median = grouped['orientation_level_num'].median()
    orientation_level_min = grouped['orientation_level_num'].min()

    # Create a dictionary to store 'daylight_label' for each apartment
    apartment_daylight_labels = {}
    apartment_view_labels = {}
    apartment_orientation_labels = {}

    # Step 3: Apply the function to each apartment group and update the dictionary
    for group_name, group_data in grouped:
        daylight_label = find_daylight_label(group_data)
        apartment_daylight_labels[group_name] = daylight_label

        view_label = find_view_label(group_data)
        apartment_view_labels[group_name] = view_label

        orientation_label = find_orientation_label(group_data)
        apartment_orientation_labels[group_name] = orientation_label

    # Step 4: Update 'daylight_label' in the original DataFrame for the rooms that belong to the apartment
    df_pred_design['daylight_label'] = df_pred_design['apartment_id'].map(apartment_daylight_labels)
    df_pred_design['view_label'] = df_pred_design['apartment_id'].map(apartment_view_labels)
    df_pred_design['orientation_label'] = df_pred_design['apartment_id'].map(apartment_orientation_labels)


    # Find overall apartment label
    df_pred_design['daylight_label_num'] = df_pred_design['daylight_label'].map(mapping_daylight_label)
    df_pred_design['view_label_num'] = df_pred_design['view_label'].map(mapping_view_label)
    df_pred_design['orientation_label_num'] = df_pred_design['orientation_label'].map(mapping_orientation_label)

    df_pred_design['overall_label_num'] = df_pred_design.apply(
        lambda row: find_overall_label_num(row['daylight_label_num'], row['view_label_num'], row['orientation_label_num']),
        axis=1
    )

    df_pred_design['overall_label'] = df_pred_design.apply(
    lambda row: find_overall_label(row['overall_label_num']),
    axis=1
    )

    # Drop the specified columns
    columns_to_drop = ['daylight_level_num', 'view_level_num', 'orientation_level_num', 'daylight_label_num', 'view_label_num', 'orientation_label_num', 'overall_label_num']
    df_pred_design = df_pred_design.drop(columns=columns_to_drop, axis=1)

    return df_pred_design

## Post-processing plots

### Roomtypes plot

In [64]:
def bbox_rect_app(gdf_app):
  #Create a dataframe with only rooms and create a small buffer for geometry
  gdf_rooms = gdf_app.loc[~gdf_app["entity_subtype"].isin(['WINDOW', 'DOOR', 'OUTSIDE_DOOR', 'FLOOR', 'FLOOR_OUTDOOR_SPACE', 'OUTDOOR_SPACE'])]
  gdf_rooms["geometry1"] = gdf_rooms["geometry"].buffer(0.2)

  #Take the unary union of the buffered polygons and create the minimum rotated rectangle
  poly_rooms = gdf_rooms["geometry1"].unary_union
  rect_rooms = poly_rooms.minimum_rotated_rectangle

  #Calculate the current dimensions of the rectangle in meters
  width, height = rect_rooms.exterior.xy
  current_width = max(width) - min(width)
  current_height = max(height) - min(height)

  #Scale the rectangle tothe specified unit size = 25x25
  xfact = 25 / current_width
  yfact = 25 / current_height
  rect_rooms_buffered = affinity.scale(rect_rooms, xfact=xfact, yfact=yfact)

  #Create a bounding box around the rect_rooms_buffered with size 25x25
  rect = box(*rect_rooms_buffered.bounds)
  gdf_rect = gpd.GeoDataFrame(geometry=[rect])

  return gdf_rect

In [65]:
# Define the desired sort order as a list with corresponding colours
# Create a dictionary to map categories to their corresponding colors
category_color_roomtypes = {
                     'OTHER':         '#377080',  # midnight green
                     'BATHROOM':      '#65949F',  # blue (munsell)                      
                     'ROOM':          '#C2DBDC',  # light blue
                     'BEDROOM':       '#C2DBDC',  # light blue
                     'STUDIO':        '#C2DBDC',  # light blue
                     'VOID':          '#F1FFFA',  # mint cream

                     'LIVING_ROOM':   '#F6A96F',  # sandy brown
                     'KITCHEN':       '#F7D08A',  # sunset / light yellow
                     'DINING':        '#f7bd7d',  # fawn / light orange
                     'STOREROOM':     '#F3DFAA',  # vanilla

                     'CIRCULATION':   '#B0647E',  # china rose
                     'CORRIDOR':      '#D7A6B3',  # orchid pink
                     'PUBLIC':        '#F3E2E7',  # lavender blush 
                     
                     'OUTDOOR_SPACE': '#A9D8B0',  # tea green

                     'WALL':          '#808081',  #grey
                     'DOOR':          '#d3d3d3',  #lightgrey
                     'OUTSIDE_DOOR':  '#94BDAA',  #cambridge blue
                     'WINDOW':        '#B5E3F1'   #light blue
                     }

# Extract the categories from the data and map them to their corresponding colors
categories_roomtypes = list(category_color_roomtypes.keys())
colors_roomtypes = list(category_color_roomtypes.values())

# Create a color map from the used colors
cmap_roomtypes = ListedColormap(colors_roomtypes)

In [66]:
# Color map for background of prediction plots
# Create a dictionary to map categories to their corresponding colors
category_color_backround = {'FLOOR':                '#F3F2F1',
                           'FLOOR_OUTDOOR_SPACE':   '#ECEAE9',
                           'DOOR':                  '#D3D3D3',
                           'OUTSIDE_DOOR':          '#D3D3D3',
                           'WINDOW':                '#D3D3D3',
                           }

# Extract the categories from the data and map them to their corresponding colors
categories_background = list(category_color_backround.keys())
colors_background = [category_color_backround[category] for category in category_color_backround]

# Create a color map from the used colors
cmap_background = ListedColormap(colors_background)

In [67]:
def plot_app_roomtypes(df_casestudy, app_id, path_casestudy):
    df_app = df_casestudy.loc[df_casestudy['apartment_id'] == app_id]
    gdf_app = gpd.GeoDataFrame(df_app, geometry='geometry', crs=None)
    
    if not gdf_app.empty:
        #create a rotated rectangular bounding box around apartment with offset 
        gdf_rect = bbox_rect_app(gdf_app)

        #Create the other geometries of the floor that fall within the bounding box
        gdf_floor = gdf_app.loc[gdf_app['entity_subtype'].isin(['FLOOR', 'FLOOR_OUTDOOR_SPACE', 'WINDOW', 'DOOR', 'OUTSIDE_DOOR'])]
        gdf_windows = gdf_app.loc[gdf_app['entity_subtype'].isin(['WINDOW', 'DOOR', 'OUTSIDE_DOOR'])]
        gdf_livingspaces = gdf_app.loc[~gdf_app['entity_subtype'].isin(['FLOOR', 'FLOOR_OUTDOOR_SPACE', 'WINDOW', 'DOOR', 'OUTSIDE_DOOR'])]
        
        # Plotting the GeoDataFrame with a title
        fig, ax = plt.subplots(figsize=(15, 15))
        
        #Plot the rooms bounding polygon, the 25x25 limit box, and the rest of the floor
        gdf_floor.plot(column="entity_subtype", cmap=cmap_background, legend=False, ax=ax, categories=category_color_backround)
        gdf_windows.plot(column="entity_subtype", cmap=cmap_background, legend=False, ax=ax, categories=category_color_backround)      
        gdf_livingspaces.plot(column="entity_subtype", cmap=cmap_roomtypes, legend=True, ax=ax, legend_kwds={'loc': 'lower right', 'fontsize': 8}, categories=category_color_roomtypes)

        gdf_rect.boundary.plot(ax=ax, color='grey', linestyle=(5, (10, 3)), linewidth=1)
        
        # Setting appropriate axis limits to remove the white space around the boundary
        ax.set_xlim(gdf_rect.bounds['minx'].min(), gdf_rect.bounds['maxx'].max())
        ax.set_ylim(gdf_rect.bounds['miny'].min(), gdf_rect.bounds['maxy'].max())
        plt.title(f' {app_id} --- room types', fontsize=12)

        # Save the plot as an image
        folder_path = os.path.join(path_casestudy, 'CS_designs_roomtypes')
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
        image_name = f'{app_id}_roomtypes.png'
        image_path = os.path.join(folder_path, image_name)
        plt.savefig(image_path, bbox_inches='tight')
        plt.close()  # Close the plot after saving

### apartment plot prediction values

In [68]:
# cmap from red to yellow to green to visualise the prediction values in rooms
# Define your custom color points and positions
colors_pred = ['#E85F58', '#FF8F57', '#FFB54C', '#F8D66D', '#FBEB83', '#A9E794', '#90CF8E', '#69B56E', '#509151']
positions_pred_day = [0.0, 0.05, 0.1, 0.15, 0.2, 0.3, 0.5, 0.75, 1.0]
positions_pred_view = [0.0, 0.025, 0.05, 0.075, 0.15, 0.4, 0.6, 0.8, 1.0]

# Create the custom colormap using LinearSegmentedColormap
cmap_pred_day_values = mcolors.LinearSegmentedColormap.from_list('CustomCmap', list(zip(positions_pred_day, colors_pred)))
cmap_pred_view_values = mcolors.LinearSegmentedColormap.from_list('CustomCmap', list(zip(positions_pred_view, colors_pred)))

# Color map for levels of prediction plots
# Create a dictionary to map categories to their corresponding colors
category_color_levels = {'insufficient':    '#E85F58',
                           'min':           '#FFB54C',
                           'low':           '#F8D66D',
                           'med':           '#A9E794',
                           'high':          '#509151',
                           }

# Extract the categories from the data and map them to their corresponding colors
categories_levels = list(category_color_levels.keys())
colors_levels = [category_color_levels[category] for category in category_color_levels]

# Create a color map from the used colors
cmap_levels = ListedColormap(colors_levels)

# Color map for labels
category_color_labels = {'F':           '#E85F58',
                         'E':           '#FFB54C',
                         'D':           '#F8D66D',
                         'C':           '#A9E794',
                         'B':           '#90CF8E',
                         'A':           '#509151',
                           }

In [69]:
def plot_pred_values(df_casestudy, df_pred_design, app_id, path_casestudy):
    df_app = df_casestudy.loc[df_casestudy['apartment_id'] == app_id]
    gdf_app = gpd.GeoDataFrame(df_app, geometry='geometry', crs=None)

    columns_norm = ['daylight_Mar_norm', 'daylight_Jun_norm', 'daylight_Dec_norm', 'view_ground_norm', 'view_sky_norm']
    columns_unnorm = ['daylight_Mar', 'daylight_Jun', 'daylight_Dec', 'view_ground', 'view_sky']
    titles = ['Daylight 21st March at 12:00 - median [lx]', 'Daylight 21st June at 12:00 - median [lx]', 'Daylight 21st December at 12:00 - median [lx]', 'View to ground - p80 [%]', 'View to sky - p80 [%]']
    cmaps_preds_values = [cmap_pred_day_values, cmap_pred_day_values, cmap_pred_day_values, cmap_pred_view_values, cmap_pred_view_values]

    if not gdf_app.empty:
        #create a rotated rectangular bounding box around apartment with offset 
        gdf_rect = bbox_rect_app(gdf_app)

        #Create the other geometries of the floor that fall within the bounding box
        gdf_floor = gdf_app.loc[gdf_app['entity_subtype'].isin(['FLOOR', 'FLOOR_OUTDOOR_SPACE', 'WINDOW', 'DOOR', 'OUTSIDE_DOOR'])]
        gdf_windows = gdf_app.loc[gdf_app['entity_subtype'].isin(['WINDOW', 'DOOR', 'OUTSIDE_DOOR'])]
        gdf_otherspaces = gdf_app.loc[gdf_app['entity_subtype'].isin(['CORRIDOR', 'BATHROOM', 'OUTDOOR_SPACE'])]
        gdf_livingspaces = gdf_app.loc[~gdf_app['entity_subtype'].isin(['FLOOR', 'FLOOR_OUTDOOR_SPACE', 'CORRIDOR', 'BATHROOM', 'OUTDOOR_SPACE', 'WINDOW', 'DOOR', 'OUTSIDE_DOOR'])]
        df_livingspaces_pred = df_pred_design.loc[~df_pred_design['entity_subtype'].isin(['FLOOR', 'FLOOR_OUTDOOR_SPACE', 'CORRIDOR', 'BATHROOM', 'OUTDOOR_SPACE', 'WINDOW', 'DOOR', 'OUTSIDE_DOOR'])]
        gdf_livingspaces_pred = gdf_livingspaces.merge(df_livingspaces_pred, on=['apartment_id', 'area_id'], how='inner')
        
        for i in range(len(columns_norm)):
            column_norm = columns_norm[i]
            column_unnorm = columns_unnorm[i]
            cmap_pred_values = cmaps_preds_values[i]
            title = titles[i]

            # Plotting the GeoDataFrame with a title
            fig, ax = plt.subplots(figsize=(15, 15))
            
            #Plot the rooms bounding polygon, the 25x25 limit box, and the rest of the floor
            gdf_floor.plot(column="entity_subtype", cmap=cmap_background, legend=False, ax=ax, categories=categories_background)
            gdf_windows.plot(column="entity_subtype", cmap=cmap_background, legend=False, ax=ax, categories=categories_background)
            gdf_otherspaces.plot(column="entity_subtype", ax=ax, edgecolor="#D3D3D3", facecolor="none")
            
            # Plot the prediction values for ach living space
            gdf_livingspaces_pred.plot(column=column_norm, cmap=cmap_pred_values, ax=ax, vmax=1)

            # Iterate through the geometries and add numbers in the middle
            for idx, geom in gdf_livingspaces_pred.geometry.iteritems():
                centroid = geom.centroid
                x, y = centroid.x, centroid.y
                value = gdf_livingspaces_pred.loc[idx, column_unnorm]
                ax.text(x, y, str(value), ha='center', va='center', fontsize=10)

            gdf_rect.boundary.plot(ax=ax, color='grey', linestyle=(5, (10, 3)), linewidth=1)
            
            # Setting appropriate axis limits to remove the white space around the boundary
            ax.set_xlim(gdf_rect.bounds['minx'].min(), gdf_rect.bounds['maxx'].max())
            ax.set_ylim(gdf_rect.bounds['miny'].min(), gdf_rect.bounds['maxy'].max())
            plt.title(f' {app_id} --- {title}', fontsize=12)

            # Save the plot as an image
            folder_path = os.path.join(path_casestudy, 'CS_designs_pred_values')
            if not os.path.exists(folder_path):
                os.makedirs(folder_path)
            image_name = f'{app_id}_pred_{column_unnorm}.png'
            image_path = os.path.join(folder_path, image_name)
            plt.savefig(image_path, bbox_inches='tight')
            plt.close()  # Close the plot after saving

In [70]:
def plot_pred_levels(df_casestudy, df_pred_design, app_id, path_casestudy):
    df_app = df_casestudy.loc[df_casestudy['apartment_id'] == app_id]
    gdf_app = gpd.GeoDataFrame(df_app, geometry='geometry', crs=None)

    columns = ['daylight_level', 'view_level', 'orientation_level']
    titles = ['Daylight performance levels', 'View performance level', 'Orientation performance level']

    if not gdf_app.empty:
        #create a rotated rectangular bounding box around apartment with offset 
        gdf_rect = bbox_rect_app(gdf_app)

        #Create the other geometries of the floor that fall within the bounding box
        gdf_floor = gdf_app.loc[gdf_app['entity_subtype'].isin(['FLOOR', 'FLOOR_OUTDOOR_SPACE', 'WINDOW', 'DOOR', 'OUTSIDE_DOOR'])]
        gdf_windows = gdf_app.loc[gdf_app['entity_subtype'].isin(['WINDOW', 'DOOR', 'OUTSIDE_DOOR'])]
        gdf_otherspaces = gdf_app.loc[gdf_app['entity_subtype'].isin(['CORRIDOR', 'BATHROOM', 'OUTDOOR_SPACE'])]
        gdf_livingspaces = gdf_app.loc[~gdf_app['entity_subtype'].isin(['FLOOR', 'FLOOR_OUTDOOR_SPACE', 'CORRIDOR', 'BATHROOM', 'OUTDOOR_SPACE', 'WINDOW', 'DOOR', 'OUTSIDE_DOOR'])]
        df_livingspaces_pred = df_pred_design.loc[~df_pred_design['entity_subtype'].isin(['FLOOR', 'FLOOR_OUTDOOR_SPACE', 'CORRIDOR', 'BATHROOM', 'OUTDOOR_SPACE', 'WINDOW', 'DOOR', 'OUTSIDE_DOOR'])]
        gdf_livingspaces_pred = gdf_livingspaces.merge(df_livingspaces_pred, on=['apartment_id', 'area_id'], how='inner')
        
        for i in range(len(columns)):
            column = columns[i]
            title = titles[i]

            if i == 2:
                gdf_livingspaces = gdf_app.loc[~gdf_app['entity_subtype'].isin(['FLOOR', 'FLOOR_OUTDOOR_SPACE', 'CORRIDOR', 'BATHROOM', 'WINDOW', 'DOOR', 'OUTSIDE_DOOR'])]
                df_livingspaces_pred = df_pred_design.loc[~df_pred_design['entity_subtype'].isin(['FLOOR', 'FLOOR_OUTDOOR_SPACE', 'CORRIDOR', 'BATHROOM', 'WINDOW', 'DOOR', 'OUTSIDE_DOOR'])]
                gdf_livingspaces_pred = gdf_livingspaces.merge(df_livingspaces_pred, on=['apartment_id', 'area_id'], how='inner')

            # Plotting the GeoDataFrame with a title
            fig, ax = plt.subplots(figsize=(15, 15))
            
            #Plot the rooms bounding polygon, the 25x25 limit box, and the rest of the floor
            gdf_floor.plot(column="entity_subtype", cmap=cmap_background, legend=False, ax=ax, categories=categories_background)
            gdf_windows.plot(column="entity_subtype", cmap=cmap_background, legend=False, ax=ax, categories=categories_background)
            gdf_otherspaces.plot(column="entity_subtype", ax=ax, edgecolor="#D3D3D3", facecolor="none")
            
            # Plot the prediction values for ach living space          
            gdf_livingspaces_pred.plot(column=column, cmap=cmap_levels, categories=categories_levels, ax=ax)

            # Iterate through the geometries and add numbers in the middle
            for idx, geom in gdf_livingspaces_pred.geometry.iteritems():
                centroid = geom.centroid
                x, y = centroid.x, centroid.y
                value = gdf_livingspaces_pred.loc[idx, column]
                ax.text(x, y, str(value), ha='center', va='center', fontsize=10)

            gdf_rect.boundary.plot(ax=ax, color='grey', linestyle=(5, (10, 3)), linewidth=1)
            
            # Setting appropriate axis limits to remove the white space around the boundary
            ax.set_xlim(gdf_rect.bounds['minx'].min(), gdf_rect.bounds['maxx'].max())
            ax.set_ylim(gdf_rect.bounds['miny'].min(), gdf_rect.bounds['maxy'].max())
            plt.title(f' {app_id} --- {title}', fontsize=12)

            
            # Save the plot as an image
            folder_path = os.path.join(path_casestudy, 'CS_designs_pred_levels')
            if not os.path.exists(folder_path):
                os.makedirs(folder_path)
            image_name = f'{app_id}_pred_{column}.png'
            image_path = os.path.join(folder_path, image_name)
            plt.savefig(image_path, bbox_inches='tight')
            plt.close()  # Close the plot after saving

In [71]:
def plot_pred_labels(df_casestudy, df_pred_design, app_id, path_casestudy):
    df_app = df_casestudy.loc[df_casestudy['apartment_id'] == app_id]
    gdf_app = gpd.GeoDataFrame(df_app, geometry='geometry', crs=None)

    df_pred_app = df_pred_design.loc[df_pred_design['apartment_id'] == app_id]
    perf_labels = df_pred_app.iloc[0][['daylight_label', 'view_label', 'orientation_label', 'overall_label']].tolist()
    
    names = ['daylight_label', 'view_label', 'orientation_label', 'overall_label']
    titles = ['Daylight performance labels', 'View performance label', 'Orientation performance label', 'Overall visual comfort label']

    if not gdf_app.empty: 
        #create a rotated rectangular bounding box around apartment with offset 
        gdf_rect = bbox_rect_app(gdf_app)

        #Create the other geometries of the floor that fall within the bounding box
        gdf_floor = gdf_app.loc[gdf_app['entity_subtype'].isin(['FLOOR', 'FLOOR_OUTDOOR_SPACE', 'WINDOW', 'DOOR', 'OUTSIDE_DOOR'])]
        gdf_windows = gdf_app.loc[gdf_app['entity_subtype'].isin(['WINDOW', 'DOOR', 'OUTSIDE_DOOR'])]
        gdf_otherspaces = gdf_app.loc[gdf_app['entity_subtype'].isin(['OUTDOOR_SPACE'])]
        gdf_livingspaces = gdf_app.loc[~gdf_app['entity_subtype'].isin(['FLOOR', 'FLOOR_OUTDOOR_SPACE', 'OUTDOOR_SPACE', 'WINDOW', 'DOOR', 'OUTSIDE_DOOR'])]

        # Create one combined polygon for the full apartment
        geom_rooms = gdf_livingspaces['geometry'].tolist()
        geom_app = [geometry.buffer(0.1) for geometry in geom_rooms]
        poly_app = MultiPolygon(geom_app)
        gdf_app = gpd.GeoDataFrame(geometry=[poly_app])
        
        for i in range(len(names)):
            name = names[i]
            title = titles[i]
            perf_label = perf_labels[i]
            facecolor = category_color_labels.get(perf_label)

            # Plotting the GeoDataFrame with a title
            fig, ax = plt.subplots(figsize=(15, 15))

            #Plot the rooms bounding polygon, the 25x25 limit box, and the rest of the floor
            gdf_floor.plot(column="entity_subtype", cmap=cmap_background, legend=False, ax=ax, categories=categories_background)
            gdf_windows.plot(column="entity_subtype", cmap=cmap_background, legend=False, ax=ax, categories=categories_background)
            gdf_otherspaces.plot(column="entity_subtype", ax=ax, edgecolor="#D3D3D3", facecolor="none")

            # Plot the union of all buffered polygons
            gdf_app.plot(ax=ax, facecolor=facecolor)
            centroid = gdf_app.centroid
            x, y = centroid.x, centroid.y
            ax.text(x, y, str(perf_label), ha='center', va='center', fontsize=18)

            gdf_rect.boundary.plot(ax=ax, color='grey', linestyle=(5, (10, 3)), linewidth=1)

            # Setting appropriate axis limits to remove the white space around the boundary
            ax.set_xlim(gdf_rect.bounds['minx'].min(), gdf_rect.bounds['maxx'].max())
            ax.set_ylim(gdf_rect.bounds['miny'].min(), gdf_rect.bounds['maxy'].max())
            plt.title(f' {app_id} --- {title}', fontsize=12)

            
            # Save the plot as an image
            folder_path = os.path.join(path_casestudy, 'CS_designs_pred_labels')
            if not os.path.exists(folder_path):
                os.makedirs(folder_path)
            image_name = f'{app_id}_pred_{name}.png'
            image_path = os.path.join(folder_path, image_name)
            plt.savefig(image_path, bbox_inches='tight')
            plt.close()  # Close the plot after saving'''

# Case study

### Import trained model

In [72]:
# Define the path to the case study folder
path_casestudy = f'C:/Users/Name/OneDrive - Delft University of Technology/Building Technology-Thesis/CaseStudy/CaseStudy2/'

# Load the ML model for daylight
path_model_daylight = path_casestudy + 'Model_finetuned_daylight.h5'
ML_model_CS_daylight = keras.models.load_model(path_model_daylight)

# Load the ML model for view
path_model_view = path_casestudy + 'Model_finetuned_view.h5'
ML_model_CS_view = keras.models.load_model(path_model_view)

### Import designs

In [73]:
#Call CSV file of dataset, and import dataset using Pandas
name_CSV = 'df_casestudy2_designs.csv'
path_CSV = path_casestudy + name_CSV
Swiss_casestudy = pd.read_csv(path_CSV)

#Create dataframe with all informantion of desings
df_casestudy = pd.DataFrame(Swiss_casestudy)

In [74]:
# Create image feature for each room in each apartment design
create_image_feature(df_casestudy, path_CS_png, path_CS_npy)

In [75]:
# Create numerical features and find additional information
app_ids, area_ids, used_files_id, entity_subtypes, images_designs, num_features = make_features_labels(df_casestudy, path_CS_npy)

C:\Users\Name\AppData\Local\Temp\ipykernel_18736\487996297.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_casestudy_rooms['filename'] = df_casestudy_rooms.apply(create_filename, axis=1)
C:\Users\Name\AppData\Local\Temp\ipykernel_18736\1540615480.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_image_feature = df_image_feature.append({
C:\Users\Name\AppData\Local\Temp\ipykernel_18736\1540615480.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_image_feature = df_image_feature.append({
C:\Users\Name\AppData\Local\Temp\ipykernel_18736\1540615480.py:15: F

### Predictions

In [76]:
# make predictions with the pre-trained model
pred_daylight = ML_model_CS_daylight.predict((images_designs, num_features))
pred_view = ML_model_CS_view.predict((images_designs, num_features))

1/1 [==============================] - 5s 5s/step


### Post-process predictions

In [77]:
# Unnormalise the predictions
daylight_values, view_values, df_pred_design = unnormalise_predictions(df_casestudy, pred_daylight, pred_view, app_ids, area_ids)

C:\Users\Name\AppData\Local\Temp\ipykernel_18736\2325336820.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_casestudy_rooms.loc[:, 'view_landscape'] = df_casestudy_rooms['view_landscape_nature_mean'] + df_casestudy_rooms['view_landscape_urban_mean']


In [78]:
# Find the performance levels & labels
df_pred_design = find_performance_levels(df_pred_design)
df_pred_design = find_performance_labels(df_pred_design)
df_pred_design.head()

,apartment_id,area_id,entity_subtype,elevation,height,orientation,window_area,window_floor_ratio,view_landscape,daylight_Mar_norm,...,daylight_Dec,view_ground,view_sky,daylight_level,view_level,orientation_level,daylight_label,view_label,orientation_label,overall_label
0,Design1,100,BEDROOM,8.7,2.6,West,2.72,0.206437,3.207000,0.133730,...,130.0,0.96,1.26,min,high,min,E,A,E,D
1,Design1,101,BEDROOM,8.7,2.6,West,2.72,0.281501,3.960711,0.352977,...,310.0,1.55,2.97,low,high,min,E,A,E,D
2,Design1,102,BEDROOM,8.7,2.6,West,2.72,0.208639,3.246874,0.216232,...,202.0,1.08,1.55,min,high,min,E,A,E,D
3,Design1,105,KITCHEN,8.7,2.6,South,1.44,0.202272,1.495387,0.178919,...,170.0,0.70,1.01,min,high,low,E,A,E,D
4,Design1,107,LIVING_ROOM,8.7,2.6,East,5.36,0.290599,2.216203,0.179232,...,159.0,0.81,0.48,min,high,min,E,A,E,D


### Plots of desings

In [79]:
#Find list of unique designs 
unique_apartment = df_casestudy['apartment_id'].unique()

# plot room types
for app_id in unique_apartment:
    plot_app_roomtypes(df_casestudy, app_id, path_casestudy)

c:\Users\Name\anaconda3\Lib\site-packages\geopandas\geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


c:\Users\Name\anaconda3\Lib\site-packages\geopandas\geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\Name\anaconda3\Lib\site-packages\geopandas\geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\Name\anaconda3\Lib\site-packages\geopandas\geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

In [80]:
# plot prediction values
for app_id in unique_apartment:
    plot_pred_values(df_casestudy, df_pred_design, app_id, path_casestudy)

c:\Users\Name\anaconda3\Lib\site-packages\geopandas\geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\Name\AppData\Local\Temp\ipykernel_18736\315708473.py:40: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for idx, geom in gdf_livingspaces_pred.geometry.iteritems():
C:\Users\Name\AppData\Local\Temp\ipykernel_18736\315708473.py:40: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for idx, geom in gdf_livingspaces_pred.geometry.iteritems():
C:\Users\Name\AppData\Local\Temp\ipykernel_18736\315708473.py:40: FutureWarning: iteritems is deprecated and will be removed in a future version. 

In [81]:
# plot prediction levels
for app_id in unique_apartment:
    plot_pred_levels(df_casestudy, df_pred_design, app_id, path_casestudy)

c:\Users\Name\anaconda3\Lib\site-packages\geopandas\geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


C:\Users\Name\AppData\Local\Temp\ipykernel_18736\2581670883.py:41: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for idx, geom in gdf_livingspaces_pred.geometry.iteritems():
C:\Users\Name\AppData\Local\Temp\ipykernel_18736\2581670883.py:41: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for idx, geom in gdf_livingspaces_pred.geometry.iteritems():
C:\Users\Name\AppData\Local\Temp\ipykernel_18736\2581670883.py:41: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for idx, geom in gdf_livingspaces_pred.geometry.iteritems():
c:\Users\Name\anaconda3\Lib\site-packages\geopandas\geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

In [82]:
# plot room types
for app_id in unique_apartment:
    plot_pred_labels(df_casestudy, df_pred_design, app_id, path_casestudy)

c:\Users\Name\anaconda3\Lib\site-packages\geopandas\geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\Name\anaconda3\Lib\site-packages\geopandas\geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\Name\anaconda3\Lib\site-packages\geopandas\geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 